In [ ]:
import os

os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

import bilby
import corner
import h5py
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import numpy.lib.recfunctions as rfn
from pesummary.core.plots.publication import reverse_triangle_plot

from thesis_utils.gw import get_cbc_parameter_labels
from thesis_utils.plotting import set_plotting, save_figure, get_default_corner_kwargs

set_plotting()

In [ ]:
gwtc1_file = "../gwtc-1_sample_release/GW150914_GWTC-1.hdf5"

with h5py.File(gwtc1_file, "r") as f:
    print(f.keys())
    gwtc1_Pv2_posterior = f["IMRPhenomPv2_posterior"][()]
    gwtc1_overall_posterior = f["Overall_posterior"][()]

In [ ]:
gwtc2p1_file = "../gwtc-2.1_sample_release/IGWN-GWTC2p1-v2-GW150914_095045_PEDataRelease_mixed_nocosmo.h5"
with h5py.File(gwtc2p1_file, "r") as f:
    print(f["C01:IMRPhenomXPHM"].keys())
    gwtc2p1_XPHM_posterior = f["C01:IMRPhenomXPHM/posterior_samples"][()]

In [ ]:
gwtc1_overall_posterior = rfn.append_fields(
    gwtc1_overall_posterior,
    ["chirp_mass", "mass_ratio"],
    [
        bilby.gw.conversion.component_masses_to_chirp_mass(
            gwtc1_overall_posterior["m1_detector_frame_Msun"],
            gwtc1_overall_posterior["m2_detector_frame_Msun"],
        ),
        bilby.gw.conversion.component_masses_to_mass_ratio(
            gwtc1_overall_posterior["m1_detector_frame_Msun"],
            gwtc1_overall_posterior["m2_detector_frame_Msun"],
        ),
    ],
    dtypes=["f8", "f8"],
)

In [ ]:
gwtc1_Pv2_posterior = rfn.append_fields(
    gwtc1_Pv2_posterior,
    ["chirp_mass", "mass_ratio"],
    [
        bilby.gw.conversion.component_masses_to_chirp_mass(
            gwtc1_Pv2_posterior["m1_detector_frame_Msun"],
            gwtc1_Pv2_posterior["m2_detector_frame_Msun"],
        ),
        bilby.gw.conversion.component_masses_to_mass_ratio(
            gwtc1_Pv2_posterior["m1_detector_frame_Msun"],
            gwtc1_Pv2_posterior["m2_detector_frame_Msun"],
        ),
    ],
    dtypes=["f8", "f8"],
)

In [ ]:
result_files = {
    "GWTC-2.1": "/home/michael.williams/git_repos/nessai-gwtc-1/analysis/IMRPhenomPv2/outdir_nessai_gwtc_1_GW150914_cal_reweight_Pv2/final_result/nessai_gwtc_1_GW150914_data0_1126259462-391_analysis_H1L1_merge_result.hdf5",
    "marg": "/home/michael.williams/git_repos/nessai-gwtc-1/analysis/original_priors/outdir_nessai_gwtc_1_GW150914_cal_reweight_gw/final_result/nessai_gwtc_1_GW150914_data0_1126259462-391_analysis_H1L1_merge_result.hdf5",
    "no-marg": "/home/michael.williams/git_repos/nessai-gwtc-1/analysis/original_priors/outdir_nessai_gwtc_1_GW150914_no_phase_marg/final_result/nessai_gwtc_1_GW150914_data0_1126259462-391_analysis_H1L1_merge_result.hdf5",
}

In [ ]:
results = {k: bilby.core.result.read_in_result(rf) for k, rf in result_files.items()}

In [ ]:
legend_elements = [
    Line2D([0], [0], color="k", ls="-", label=r"GWTC-1"),
    Line2D([0], [0], color="k", ls="--", label=r"GWTC-2.1"),
    Line2D([0], [0], color="C0", ls="-", label=r"\texttt{nessai} - GWTC-2.1 + Marg."),
    Line2D([0], [0], color="C1", ls="-", label=r"\texttt{nessai} - GWTC-1 + Marg."),
    Line2D([0], [0], color="C2", ls="-", label=r"\texttt{nessai} - GWTC-1 + No marg."),
]

In [ ]:
corner_kwargs = get_default_corner_kwargs()
corner_kwargs["plot_density"] = False
corner_kwargs["plot_datapoints"] = False
corner_kwargs["no_fill_contours"] = True
corner_kwargs["show_titles"] = False
corner_kwargs.pop("fill_contours")
corner_kwargs["levels"] = (1 - np.exp(-0.5), 1 - np.exp(-2), 1 - np.exp(-9 / 2.0))
corner_kwargs["quantiles"] = []
corner_kwargs["smooth"] = 0.9
corner_kwargs["bins"] = 32
fig = None

corner_kwargs["color"] = "k"
corner_kwargs["hist_kwargs"]["color"] = "k"
fig = corner.corner(
    np.array([gwtc1_Pv2_posterior["chirp_mass"], gwtc1_Pv2_posterior["mass_ratio"]]).T,
    fig=fig,
    labels=get_cbc_parameter_labels(["chirp_mass", "mass_ratio"], units=True),
    labelpad=-0.1,
    **corner_kwargs,
)

corner_kwargs["hist_kwargs"]["ls"] = "--"
fig = corner.corner(
    np.array(
        [gwtc2p1_XPHM_posterior["chirp_mass"], gwtc2p1_XPHM_posterior["mass_ratio"]]
    ).T,
    fig=fig,
    contour_kwargs=dict(linestyles=["--"]),
    **corner_kwargs,
)
corner_kwargs["hist_kwargs"]["ls"] = "-"

for i, (key, result) in enumerate(results.items()):
    colour = f"C{i}"
    corner_kwargs["color"] = colour
    corner_kwargs["hist_kwargs"]["color"] = colour
    fig = corner.corner(
        np.array(
            [
                result.posterior["chirp_mass"].to_numpy(),
                result.posterior["mass_ratio"].to_numpy(),
            ]
        ).T,
        fig=fig,
        **corner_kwargs,
    )

# axs = plt.gca()
fig.legend(
    handles=legend_elements,
    bbox_to_anchor=(1.0, 0.9),
    loc="upper right",
)
plt.show()
save_figure(fig, "GW150914_mass_comparison")

In [ ]:
corner.hist2d(
    np.random.randn(1000),
    np.random.randn(1000),
    no_fill_contours=True,
    contourf_kwargs=dict(linestyles=["--"]),
    # contour_kwargs=dict(linestyles=["--"]),
)

In [ ]:
reverse_triangle_plot(
    gwtc1_Pv2_posterior["chirp_mass"],
    gwtc1_Pv2_posterior["mass_ratio"],
)

In [ ]:
bilby.core.result.plot_multiple(
    results,
    save=False,
    parameters=["chirp_mass", "mass_ratio", "a_1", "a_2", "tilt_1", "tilt_2"],
)